<a href="https://colab.research.google.com/github/michaelherediaperez/EspecializacionEstructuras/blob/main/01_AnalisisNoLineal/Bhatti_73_LoadIncrements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio Bhatti 7.3. Load Increments

Michael Heredia Pérez - mherediap@unal.edu.co - Octubre 04 / 2021

Mismo problema 7.1. con una variación: la ecuación diferencial definida como: 

$$\frac{d}{dx}\left(u^2\frac{du}{dx}\right) + \lambda q(x) = 0$$
$$0<x<L$$

Con las siguientes condiciones de frontera:

$$u(0) = 1;$$
$$\left(u^2\frac{du}{dx}\right)_{x=L} = \lambda p;$$

Usando valores nominales $p=1/2$ y $q=1$ y llegando a $\lambda=4$ se llega a la ED inicial.



## Solución



In [83]:
# Iportación de librerías.
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math

### Parámetros del modelo

Algunos parámetros geométricos son:

In [84]:
nefs = 3        # Número de elementos finitos.
nnds = nefs+1   # Número de nodos.
ngdl = nnds     # Número de grados de libertad.
L = 1           # Rango superior del dominio.

# Posición de los nodos (x1, x, x3, x4)
xnod = np.linspace(0, L, nnds)  

Parámetros de carga y condiciones de frontera


In [85]:
# Carga axial distribuida.
q = 1

# Vector de cargas puntuales.
Rp = np.zeros(ngdl)

# Vector de incrementos de carga.
lambdas = [2, 4]

Intruduciendo las condiciones de frontera.

In [86]:
EBC = 1    # Essential Boundary Condition u1 = 1.
NBC = 1/2    # Non-essential Boundary Condition.

### Solución por Newton-Raphson

Dado que es un problema no lineal, se puede solucionar mediante la repetición de series de análisis lineales para cantidades incrementales, así que se aplica el método de Newton-Raphson.

Se conoce que $u_1 = 1$ por EBC, se construye un vector de desplazamientos iniciales $d^{(0)}$ tal que se respete dicha condición: 

In [87]:
# Vctor inicial (suposición) de desplazamientos d^(0)
d_i = np.array([EBC, 2, 2, 2], dtype=float)   

display(Math(r'd^{(0)} = '))
print(d_i)

<IPython.core.display.Math object>

[1. 2. 2. 2.]


Durante el calculo iterativo, el vector de fuerzas externas no cambiará, así que se puede calcualr fuera del ciclo

In [88]:
Re_nbc = np.array([0, 0, 0,  NBC])

display(Math(r'R_{E,NBC} ='))
print(Re_nbc)

<IPython.core.display.Math object>

[0.  0.  0.  0.5]


Durante el calculo iterativo, el vector de fuerzas externas no cambiará, así que se puede calcular fuera del ciclo

In [89]:
# Reservo memoria: vector incompleto de fuerzas nodales externas.
Re_ = np.zeros(ngdl)    

for e in range(nefs):
    
    # Coordenadas globales de los nodos:
    x1 = xnod[e]; x2 = xnod[e+1]
    # Longitud del elemento:
    l = x2-x1

    # Vector de cargas externas local para ele elemento e:
    # re = rq + rp
    re = np.array([ q*l/2, q*l/2 ]) + np.array([ Rp[e], Rp[e+1] ])

    idx = np.array([e, e+1])
    Re_[idx] += re

# Vector de fuerzas externas final.
Re = Re_nbc + Re_
# Debido a la condición EBC, elimino el primer valor.
Re = np.delete(Re, 0) 
# Norma de Re al cuadrado.
norm_Re2 = np.sum(Re**2)

In [90]:
display(Math(r'R_{E sin NBC} ='))
print(Re_.round(5))

print("\nVector de fuerzas externas ensamblado")
display(Math(r'R_{E} ='))
print(Re.round(5))

<IPython.core.display.Math object>

[0.16667 0.33333 0.33333 0.16667]

Vector de fuerzas externas ensamblado


<IPython.core.display.Math object>

[0.33333 0.33333 0.66667]


Se hace un ciclo iterativo para solucionar el problema:

In [91]:
NIT = 3 # Número de iteraciones a realizar.
NIN = 2 # Número de incrementos de carga a tener en cuenta.

# Por cada incremento de carga:
for lam in lambdas:
    
    # Vector de fuerzas externas escalado por el factor de incremento.
    lambdaRe = lam*Re
    
    print('='*80 + "\nIncremento lambda= ", str(lam))

    # Por cada iteración:
    for itr in range(NIT):
        
        # Reservo memoria:    
        Kt = np.zeros((ngdl, ngdl)) # Matriz tangencial.
        Ri = np.zeros(ngdl)         # Vector de fuerzas internas. 

        # Calculo para cada EF.
        for e in range(nefs):

            # Desplazamientos en los nodos locales del elemento.
            u1 = d_i[e]; u2 = d_i[e+1] 
            # Coordenadas globales de los nodos:
            x1 = xnod[e]; x2 = xnod[e+1]
            # Longitud del elemento:
            l = x2-x1

            # Matriz tangencial del elemento e en la interación i
            kt_i = np.array([
                            [ u1**2/l, -u2**2/l],
                            [-u1**2/l,  u2**2/l]
                            ])
            
            # Vector de fuerzas internas.
            ri_i = np.array([(u1**3 - u2**3)/(3*l), -(u1**3 - u2**3)/(3*l)])
            
            # gdl que interceden.
            idx = np.array([e, e+1])
            
            # Ensamblaje de la matriz tangencial global.
            Kt[np.ix_(idx, idx)] += kt_i
            # Ensamblaje del vector de fuerzas internas.
            Ri[idx] += ri_i

        # Simplificaciones debido a la condición EBC.
        Kt = np.delete(Kt, 0, axis=0)
        Kt = np.delete(Kt, 0, axis=1)
        Ri = np.delete(Ri, 0)

        # Calculo del residuo R = Re - Ri
        R = lambdaRe-Ri

        # Norma de R al cuadrado.
        norm_R2 = np.sum(R**2)

        # Criterio de convergencia:
        CNR = norm_R2 / (1+lam**2*norm_Re2)

        # Solución de las ecuacioens incremetales:
        delta_d = np.linalg.solve(Kt, R)

        # Resultados.
        print('-'*80 + "\nIteración ", str(itr+1))
        display(Math(r'K_T^{} ='.format(itr)));              print(Kt.round(4))
        display(Math(r'R_I^{} ='.format(itr)));              print(Ri.round(5))
        display(Math(r'R^{} ='.format(itr)));                print(R.round(5))
        display(Math(r'\frac{\|R\|^2}{1 + \| R_E\|^2} = ')); print(CNR)
        
        # Comprobación del CNR.
        if abs(CNR - 0.001)<0.001:
            # Se deja el valor anterior.
            print("Se ha llegado a una buena convergencia.")
            display(Math(r'd^{} ='.format(itr+1))); print(d_i.round(5))
        else:
            d_i[1:] += delta_d
            display(Math(r'd^{} ='.format(itr+1))); print(d_i.round(5))      

Incremento lambda=  2
--------------------------------------------------------------------------------
Iteración  1


<IPython.core.display.Math object>

[[ 24. -12.   0.]
 [-12.  24. -12.]
 [  0. -12.  12.]]


<IPython.core.display.Math object>

[7. 0. 0.]


...

[ 0.01246 -0.01032 -0.02323]


<IPython.core.display.Math object>

6.86920696916803e-05
Se ha llegado a una buena convergencia.


<IPython.core.display.Math object>

[1.      1.85222 2.18194 2.35602]
